## Install dependencies

In [ ]:
!pip -q install git+https://github.com/huggingface/transformers
!pip -q install accelerate>=0.12.0

## Gradio

In [ ]:
# install dependencies
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!apt-get install ffmpeg
!pip install gradio
!pip install xformers

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.6 MB/s eta 0:00:00

In [ ]:
import gradio as gr
from transformers import *
import torch

# 그라디오 페이지 테마 설정
theme = gr.themes.Monochrome(
    primary_hue="indigo",
    secondary_hue="blue",
    neutral_hue="slate",
    radius_size=gr.themes.sizes.radius_sm,
    font=[gr.themes.GoogleFont("Open Sans"), "ui-sans-serif", "system-ui", "sans-serif"],
)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 6.5 MB/s eta 0:00:00


## Dolly

In [ ]:
import torch
from transformers import pipeline

# 사전학습된 토크나이저와 모델 불러오기
model_name = "databricks/dolly-v2-2-8b"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_8bit=False, trust_remote_code=True)

end_key_token_id = tokenizer.encode("### End")[0]

# Dolly 파이프라인 생성
instruct_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer,pad_token_id=tokenizer.pad_token_id, eos_token_id=end_key_token_id)


loading file vocab.json from cache at None
loading file merges.txt from cache at None
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--databricks--dolly-v2-2-8b/snapshots/877db3ed12a3086500d144b9ef74e469b107a041/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--databricks--dolly-v2-2-8b/snapshots/877db3ed12a3086500d144b9ef74e469b107a041/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--databricks--dolly-v2-2-8b/snapshots/877db3ed12a3086500d144b9ef74e469b107a041/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--databricks--dolly-v2-2-8b/snapshots/877db3ed12a3086500d144b9ef74e469b107a041/config.json
Model config GPTNeoXConfig {
  "_name_or_path": "databricks/dolly-v2-2-8b",
  "architectures": [
    "GPTNeoXForCausalLM"
  ],
  "bos_token_

In [ ]:
# 함수 설정

# 주어진 토큰으로부터 대답을 생성하는 함수
def generate(instruction):
    input_ids = tokenizer.encode(instruction, return_tensors="pt")
    input_ids = input_ids.to(model.device)  # Move input_ids to the same device as the model
    generated_output = model.generate(input_ids, max_length=256,pad_token_id=tokenizer.pad_token_id, eos_token_id=end_key_token_id)
    dd = tokenizer.decode(generated_output[0])
    return dd


examples = [
    "Instead of making a peanut butter and jelly sandwich, what else could I combine peanut butter with in a sandwich? Give five ideas",
    "How do I make a campfire?",
    "Write me a tweet about the release of Dolly 2.0, a new LLM"
]

# 예시문장 처리 함수
def process_example(args):
    for x in generate(args):
        pass
    return x


## Implementation

In [ ]:
# 그라디오로 dolly 모델 구현
css = ".generating {visibility: hidden}"

with gr.Blocks(theme=theme, analytics_enabled=False, css=css) as demo:
    with gr.Column():
        with gr.Row():
            with gr.Column(scale=3):
                instruction = gr.Textbox(placeholder="Enter your question here", label="Question", elem_id="q-input")

                with gr.Box():
                    gr.Markdown("**Answer**")
                    output = gr.Markdown(elem_id="q-output")
                submit = gr.Button("Generate", variant="primary")
                gr.Examples(
                    examples=examples,
                    inputs=[instruction],
                    cache_examples=False,
                    fn=process_example,
                    outputs=[output],
                )


    submit.click(generate, inputs=[instruction], outputs=[output])
    instruction.submit(generate, inputs=[instruction], outputs=[output])

demo.queue(concurrency_count=1).launch(debug=True)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 23>:23                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'gr' is not defined